In [ ]:
import pandas as pd
import numpy as np
from pykrige import OrdinaryKriging3D
from pykrige.compat import GridSearchCV
from pykrige.rk import Krige
import pickle

In [ ]:
# read relevant data
obs_file = '../DATA/07_day_of_observation.csv'
obs = pd.read_csv(obs_file, delimiter=';')

In [ ]:
# 3D Kriging-CV
# Parameters to search with
param_dict3d = {'method': ['ordinary3d'],
                'variogram_model': ['linear', 'power', 'gaussian', 'spherical'],
                'nlags':[6,7,8],
                'weight': [True, False]
                }
# create estimator
estimator = GridSearchCV(Krige(), param_dict3d, verbose=True, cv=5)

# coordinate-values-pairs to use
X = obs[['east','north','up']].values
y = obs['value'].values

# run the gridsearch
estimator.fit(X=X, y=y)
if hasattr(estimator, 'best_score_'):
    print('best_score R2 = {:.3f}'.format(estimator.best_score_))
    print('best_params = ', estimator.best_params_)

print('\nCV results::')
if hasattr(estimator, 'cv_results_'):
    for key in ['mean_test_score', 'mean_train_score',
                'param_method', 'param_variogram_model']:
        print(' - {} : {}'.format(key, estimator.cv_results_[key]))

In [ ]:
# extract df-columns
x = obs['east'].values
y = obs['north'].values
z = obs['up'].values
values = obs['value'].values

In [ ]:
# prepare model for export
params = estimator.best_params_
model = OrdinaryKriging3D(x, y, z, values, 
                          variogram_model=params['variogram_model'], 
                          nlags=params['nlags'], 
                          weight=params['weight'])

In [ ]:
# save model
pkl_filename = '../DATA/KRIGING/kriging_{}_{}lags_wgt{}.pkl'.format(params['variogram_model'],
                                                                      params['nlags'], params['weight'])
with open(pkl_filename, 'wb') as file:  
    pickle.dump(model, file)